# Daftar Isi <a id='back'></a>

* [Final Project SQL](#intro)
* [Membuat Koneksi pada Database](#koneksi)
* [Mempelajari Data pada Database](#mempelajari_data)
* [Hitung jumlah buku yang dirilis setelah tanggal 1 Januari 2000](#book)
* [Hitung jumlah ulasan pengguna dan rating rata-rata untuk setiap buku](#rating_count)
* [Identifikasi penerbit yang telah menerbitkan jumlah buku terbanyak dengan jumlah halaman lebih dari 50](#publisher_top)
* [Identifikasi penulis dengan rating rata-rata buku tertinggi: temukan buku dengan rating minimal 50](#author_top)
* [Hitung jumlah rata-rata teks ulasan di antara pengguna yang memberi rating terhadap lebih dari 50 buku](#avg_review)


# Final Project SQL <a id='intro'></a>

󠀰Virus corona yang kehadirannya mengejutkan seluruh dunia telah mengubah rutinitas harian semua orang.󠀲󠀡󠀠󠀦󠀥󠀨󠀧󠀢󠀳󠀰 Kini, warga kota tidak lagi menghabiskan waktu luangnya di luar rumah seperti pergi ke kafe dan mal. Mereka lebih sering berada di rumah, menghabiskan waktunya dengan membaca buku.󠀲󠀡󠀠󠀦󠀥󠀨󠀧󠀣󠀳󠀰 Hal ini pun mendorong perusahaan startup untuk mengembangkan aplikasi baru bagi para pecinta buku.󠀲󠀡󠀠󠀦󠀥󠀨󠀧󠀤󠀳

󠀰Saya telah diberi sebuah database dari salah satu perusahaan yang bersaing dalam industri ini.󠀲󠀡󠀠󠀦󠀥󠀨󠀧󠀥󠀳󠀰 Database tersebut berisi data tentang buku, penerbit, penulis, serta rating dan ulasan pelanggan atas buku terkait.󠀲󠀡󠀠󠀦󠀥󠀨󠀧󠀦󠀳󠀰 Informasi ini akan digunakan dalam membuat penawaran harga untuk sebuah produk baru.

Tujuan dari proyek ini adalah mencari informasi yang dibutuhkan perusahaan untuk membantu mengembangkan aplikasi untuk pecinta buku, yang mana di proyek ini saya akan menganalisis data pada database dan menjawab permintaan dari pihak perusahaan terkait.

## Membuat Koneksi pada Database <a id='koneksi'></a>

In [1]:
# impor library yang diperlukan
import pandas as pd
from sqlalchemy import create_engine

# membuat koneksi dengan database
db_config = {'user': xxxx,         # nama pengguna
             'pwd': xxxx, # kata sandi
             'host': xxxx,
             'port': xxxx,              # port koneksi
             'db': xxxx}          # nama database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

#membuat fungsi menjalankan query
def run_sql(query_x):
    df_x = pd.io.sql.read_sql(query_x, con = engine)
    return df_x

## Mempelajari Data pada Database <a id='mempelajari_data'></a>

In [2]:
books =''' SELECT *
            FROM books
        '''

run_sql(books).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
authors =''' SELECT *
            FROM authors
        '''

run_sql(authors).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
ratings =''' SELECT *
            FROM ratings
        '''

run_sql(ratings).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
reviews =''' SELECT *
            FROM reviews
        '''

run_sql(reviews).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
publishers =''' SELECT *
            FROM publishers
        '''

run_sql(publishers).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


data-data di dalam database dibagi menjadi 5 table, yang mana masing-masing table bisa di gabungkan berdasarkan id nya melalui table books

## Hitung jumlah buku yang dirilis setelah tanggal 1 Januari 2000.󠀲󠀡󠀠󠀦󠀥󠀩󠀠󠀩󠀳 <a id='book'></a>

In [7]:
query = ''' SELECT count(distinct title)
FROM books
WHERE publication_date > '2000-01-01'
'''

run_sql(query)

,count
0,818


In [8]:
#total buku
query = ''' SELECT count(distinct title)
FROM books
'''

run_sql(query)

,count
0,999


terdapat 818 buku yang dirilis setelah 01 januari 2000 dari total 999 buku yang tercatat di database

## Hitung jumlah ulasan pengguna dan rating rata-rata untuk setiap buku <a id='rating_count'></a>

In [10]:
query= '''
SELECT
    books.title,
    SUBQ.avg_rating,
    SUBQ.review_count

FROM
    (SELECT
        reviews.book_id as book_id,
        COUNT (distinct reviews.review_id) as review_count,
        AVG (ratings.rating) as avg_rating
    FROM
        reviews INNER JOIN ratings ON ratings.book_id = reviews.book_id
    GROUP BY
        reviews.book_id
    ORDER BY
        review_count DESC,
        avg_rating DESC) as SUBQ
INNER JOIN books ON SUBQ.book_id = books.book_id
ORDER BY
        review_count DESC,
        avg_rating DESC'''

run_sql(query)

,title,avg_rating,review_count
0,Twilight (Twilight #1),3.662500,7
1,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
2,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
3,The Book Thief,4.264151,6
4,The Glass Castle,4.206897,6
...,...,...,...
989,Death: The High Cost of Living,3.000000,1
990,Moo Baa La La La!,3.000000,1
991,1 000 Places to See Before You Die,2.500000,1
992,History of Beauty,2.500000,1


jika dilihat data diatas, buku Twilight (Twilight #1) sepertinya bukan buku yang bagus menurut review pengguna. perusahaan startup yang mengembangkan aplikasi untuk pecinta buku sepertinya bisa menyediakan buku yang setidaknya memiliki rating 4

## Identifikasi penerbit yang telah menerbitkan jumlah buku terbanyak dengan jumlah halaman lebih dari 50 <a id='publisher_top'></a>

In [11]:
query = ''' SELECT distinct publishers.publisher, count(books.title) over (partition by publishers.publisher) as books,
                    min(books.num_pages) over (partition by publishers.publisher) as min_pages
FROM books LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
---GROUP BY authors.author, books.num_pages
ORDER BY books desc
'''

run_sql(query).head(10)

,publisher,books,min_pages
0,Penguin Books,42,80
1,Vintage,31,120
2,Grand Central Publishing,25,204
3,Penguin Classics,24,70
4,Ballantine Books,19,272
5,Bantam,19,214
6,Berkley,17,144
7,Berkley Books,14,194
8,St. Martin's Press,14,132
9,Delta,13,276


berdasarkan data diatas, penerbit Penguin Books dan Vintage adalah penerbit yang lebih banyak menerbitkan buku baru dibandingkan penerbit lain

## Identifikasi penulis dengan rating rata-rata buku tertinggi: temukan buku dengan rating minimal 50 <a id='author_top'></a>

In [14]:
query='''
SELECT
    authors.author,
    AVG(SUBQ2.avg_rating) as final_avg
FROM
    (SELECT
    books.title,
    books.author_id,
    SUBQ1.avg_rating
    FROM
    (SELECT
        book_id,
        COUNT(rating_id) as rating_cnt,
        AVG(rating) as avg_rating
    FROM
        ratings
    GROUP BY
        book_id
    HAVING
        COUNT(rating_id) > 50) as SUBQ1
    INNER JOIN books on books.book_id = SUBQ1.book_id) as SUBQ2
    
INNER JOIN authors on authors.author_id = SUBQ2.author_id
GROUP BY
    author
ORDER BY
    final_avg DESC'''
run_sql(query).head(10)

,author,final_avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


terdapat 29 buku yang memiliki rating rata-rata 5.0, ini berarti setidaknya 1 penulis teratas memiliki 1 buku dengan rating 5.0. perusahaan startup sudah seharusnya menyediakan buku dengan rating tinggi seperti ini

## Hitung jumlah rata-rata teks ulasan di antara pengguna yang memberi rating terhadap lebih dari 50 buku. <a id='avg_review'></a>

In [18]:
query= '''SELECT
    ROUND(AVG(SUBQ.text_reviews_count), 2) as avg_text_reviews_count
FROM
    (SELECT
        COUNT(distinct reviews.review_id) as text_reviews_count
    FROM
        reviews INNER JOIN ratings ON reviews.username = ratings.username
    GROUP BY
        ratings.username
    HAVING
        COUNT(distinct ratings.rating_id) > 50) as SUBQ'''
run_sql(query)

,avg_text_reviews_count
0,24.33


Pada pengguna yang sudah memberikan lebih dari 50 rating, mereka rata-rata memberikan review terhadap 24 buku